## Image Classification Using PCA and Gaussian Kernel PCA

### Importing necessary libraries

In [1]:
# Import necessary libraries
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.linear_model import Perceptron
from scipy.io import loadmat
import h5py
from sklearn.metrics import zero_one_loss
from scipy import sparse
from scipy.sparse.linalg import svds, LinearOperator
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs
import warnings
warnings.filterwarnings("ignore")

### Initialize necessary functions

In [2]:
def loadmat_v73(file_name):
    """
    Loads MATLAB v7.3 format file and returns the data as a dictionary.

    Parameters:
    - file_name: Name of the MATLAB file to load.

    Returns:
    - data: Dictionary containing the data from the MATLAB file.
    """

    with h5py.File(file_name, 'r') as f:
        data = {key: np.array(f[key]) for key in f.keys()}
    return data


def PCA_sparse(X, d):
    """
    Performs dimensionality reduction using Sparse Principal Component Analysis (PCA) on a data matrix.

    Parameters:
    - X: Data matrix where each row represents an observation and each column represents a feature.
    - d: The desired reduced dimension.

    Returns:
    - Y: The dimensionality-reduced data matrix.
    - eigVector: The eigenvectors corresponding to the principal components.
    - eigValue: The eigenvalues associated with the principal components.
    """

    X = sparse.csr_matrix(X) # convert X to sparse matrix if it is not already


    # eigenvalue analysis
    X_mean = X.mean(axis=0)  # calculate mean
    X = X - X_mean  # centering

    def matvec(v):
        v = v.reshape(-1, 1)  # reshape the input vector
        return (X.T @ (X @ v)).ravel()  # ravel to make sure the output is 1D
    
    # Since our operator is symmetric (i.e., X.T @ X equals its transpose), rmatvec is the same as matvec
    rmatvec = matvec
    
    Sx_op = LinearOperator((X.shape[1], X.shape[1]), matvec=matvec, rmatvec=rmatvec)

    # Adjust this line to receive the singular values and the left and right singular vectors
    U, eigValue, Vt = svds(Sx_op, k=d)  # calculate eigenvalues and eigenvectors

    IX = np.argsort(eigValue)[::-1]  # sort eigenvalues in descending order and get indices
    eigValue = eigValue[IX]
    eigVector = Vt.T[:, IX]  # Use the right singular vectors (transpose of Vt) as eigenvectors

    # normalization
    norm_eigVector = np.sqrt(np.sum(eigVector**2, axis=0))
    eigVector = eigVector / norm_eigVector

    Y = X @ eigVector  # sparse matrix multiplication

    return Y, eigVector, eigValue


def sparse_distance_matrix(X, num_neighbors=5):
    """
    Computes a sparse pair-wise distance matrix based on the data matrix using nearest neighbors.

    Parameters:
    - X: Data matrix where each row represents an observation and each column represents a feature.
    - num_neighbors: Number of nearest neighbors to consider, not including self.

    Returns:
    - D: Sparse pair-wise distance matrix.
    """
    # Ensure X is a sparse matrix
    X = sparse.csr_matrix(X)

    # Use sklearn's NearestNeighbors to find nearest neighbors
    knn = NearestNeighbors(n_neighbors=num_neighbors+1, metric='manhattan')
    knn.fit(X)
    
    # Get the distances and indices of the nearest neighbors
    distances, indices = knn.kneighbors(X)
    
    # Build the sparse distance matrix
    m, _ = X.shape
    D = sparse.lil_matrix((m, m))
    
    for i in range(m):
        D[i, indices[i]] = distances[i]

    return D.tocsr()

def kernel(X, kernel_type, para):
    """
    Computes a sparse kernel matrix based on the data matrix and specified kernel type.

    Parameters:
    - X: Data matrix where each row represents an observation and each column represents a feature.
    - kernel_type: Type of kernel, can be 'simple', 'poly', or 'gaussian'.
    - para: Parameter for computing the 'poly' kernel. For 'simple' and 'gaussian', it will be ignored.

    Returns:
    - K: Sparse kernel matrix.
    """
    # Ensure X is a sparse matrix
    X = csr_matrix(X)

    if kernel_type == 'simple':
        K = X.dot(X.transpose())
    elif kernel_type == 'poly':
        K = X.dot(X.transpose()) + 1
        K = K.power(para)
    elif kernel_type == 'gaussian':
        D = sparse_distance_matrix(X)
        D.data **= 2  # square the distances in-place
        K = sparse.csr_matrix(np.exp(-D.toarray() / (2 * para**2)))
    else:
        raise ValueError(f"Unknown kernel_type: {kernel_type}")
    
    return K

def sparse_distance_matrix_new_data(X, Y=None, num_neighbors=5):
    """
    Computes a sparse pair-wise distance matrix between X and Y using nearest neighbors.

    Parameters:
    - X: Data matrix where each row represents an observation and each column represents a feature.
    - Y: New data matrix.
    - num_neighbors: Number of nearest neighbors to consider, not including self.

    Returns:
    - D: Sparse pair-wise distance matrix.
    """
    # Ensure X and Y are sparse matrices
    X = sparse.csr_matrix(X)
    Y = sparse.csr_matrix(Y)

    # Use sklearn's NearestNeighbors to find nearest neighbors
    knn = NearestNeighbors(n_neighbors=num_neighbors+1, metric='manhattan')
    knn.fit(X)

    # Get the distances and indices of the nearest neighbors
    distances, indices = knn.kneighbors(Y)

    # Build the sparse distance matrix
    m, _ = Y.shape
    n, _ = X.shape
    D = sparse.lil_matrix((m, n))

    for i in range(m):
        D[i, indices[i]] = distances[i]

    return D.tocsr()

def kernel_new_data(Y, X, kernel_type, para):
    """
    Computes a sparse kernel matrix between new data Y and training data X based on the specified kernel type.

    Parameters:
    - Y: New data matrix.
    - X: Training data matrix where each row represents an observation and each column represents a feature.
    - kernel_type: Type of kernel, can be 'simple', 'poly', or 'gaussian'.
    - para: Parameter for computing the 'poly' kernel. For 'simple' and 'gaussian', it will be ignored.

    Returns:
    - K: Sparse kernel matrix.
    """
    # Ensure X and Y are sparse matrices
    X = sparse.csr_matrix(X)
    Y = sparse.csr_matrix(Y)

    if kernel_type == 'simple':
        K = Y.dot(X.transpose())
    elif kernel_type == 'poly':
        K = Y.dot(X.transpose()) + 1
        K = K.power(para)
    elif kernel_type == 'gaussian':
        D = sparse_distance_matrix_new_data(X, Y)
        D.data **= 2  # square the distances in-place
        D_dense = D.toarray()  # Convert sparse matrix to dense array
        K = np.exp(-D_dense / (2 * para**2))
    else:
        raise ValueError(f"Unknown kernel_type: {kernel_type}")

    return K


def compute_sigma(X, num_neighbors=5):
    """
    Computes the value of sigma used in the Gaussian kernel based on the mean distance to the nearest neighbors.

    Parameters:
    - X: Data matrix where each row represents an observation and each column represents a feature.
    - num_neighbors: Number of nearest neighbors to consider, not including self.

    Returns:
    - sigma: Value of sigma for the Gaussian kernel.
    """
    knn = NearestNeighbors(n_neighbors=num_neighbors+1, metric='euclidean')
    knn.fit(X)
    distances, _ = knn.kneighbors(X)

    # Compute the mean of the distances to the nearest neighbors
    mean_d_NN = np.mean(distances[:, 1:])

    # Compute σ based on the formula
    sigma = 5 * mean_d_NN

    return sigma

def kpca(X, d, kernel_type, para):
    """
    Performs Kernel Principal Component Analysis (KPCA) on a data matrix X.

    Parameters:
    - X: Data matrix where each row represents an observation and each column represents a feature.
    - d: Reduced dimension.
    - kernel_type: Type of kernel, can be 'simple', 'poly', or 'gaussian'.
    - para: Parameter for computing the 'poly' and 'gaussian' kernel. For 'simple', it will be ignored.

    Returns:
    - Y: Dimensionality-reduced data.
    - eigVector: Eigen-vectors, which can be used for pre-image reconstruction.
    - eigValue: Eigenvalues.
    """
    
    if kernel_type not in ['simple', 'poly', 'gaussian']:
        print(f"\nError: Kernel type {kernel_type} is not supported. \n")
        return None, None, None
    
    if kernel_type == 'gaussian':
        sigma = compute_sigma(X)
        #sigma =para
        para = sigma

    N = X.shape[0]

    # Compute the kernel matrix
    K0 = kernel(X, kernel_type, para).toarray()  # convert to dense matrix
    oneN = np.ones((N, N)) / N
    K = K0 - oneN.dot(K0) - K0.dot(oneN) + oneN.dot(K0).dot(oneN)


    # Perform eigenvalue analysis
    eigValue, eigVector = eigs(K / N, k=d, which='LM')
    eigValue = eigValue.real
    eigVector = eigVector.real

    # Normalize the eigenvectors
    norm_eigVector = np.sqrt(np.sum(eigVector ** 2, axis=0))
    eigVector = eigVector / norm_eigVector

    # Perform dimensionality reduction
    Y = K0.dot(eigVector)

    return Y, eigVector, eigValue

### Dimensionality Reduction and Classification using PCA and Kernel PCA

In [3]:
if __name__ == "__main__":

    # Load the dataset
    data = loadmat_v73('./Dataset/YaleFaceData.mat')

    # Split the dataset into train and test sets
    train_x, train_t, test_x, test_t = data['train_x'], data['train_t'], data['test_x'], data['test_t']

    d = 9  # Initialize the reduced dimensions

    # Standard PCA
    print('Performing standard PCA...')

    # Perform PCA on the training data
    train_PCA, eigVector, _ = PCA_sparse(train_x.T, d)
    eigVector = eigVector[:, :d]

    # Transform the test data using the PCA eigenvectors
    test_PCA = np.dot(test_x.T, eigVector)
    
    # Classification using Perceptron
    classifier = Perceptron()
    classifier.fit(train_PCA, train_t.ravel())

    # Calculate error rates for PCA on training and testing data
    error_PCA_train = zero_one_loss(train_t.T.ravel(), classifier.predict(train_PCA))
    error_PCA_test = zero_one_loss(test_t.T.ravel(), classifier.predict(test_PCA))

    print(f'error rate of PCA on training data: {error_PCA_train}')
    print(f'error rate of PCA on testing data: {error_PCA_test}')
    print('\n')

    # kPCA
    print('Performing Kernel PCA...')

    # Perform Kernel PCA on the training data
    train_kPCA, eigVector_kPCA, _ = kpca(train_x.T, d, 'gaussian', para=22546)
    eigVector_kPCA = eigVector_kPCA[:, :d]

    # Transform the test data using the Kernel PCA eigenvectors
    test_kPCA = kernel_new_data(test_x.T, train_x.T, 'gaussian', para=22546).dot(eigVector_kPCA)

    # Classification using Perceptron
    classifier_kPCA = Perceptron()
    classifier_kPCA.fit(train_kPCA, train_t.ravel())

    # Calculate error rates for kPCA on training and testing data
    error_kPCA_train = zero_one_loss(train_t.T.ravel(), classifier_kPCA.predict(train_kPCA))
    error_kPCA_test = zero_one_loss(test_t.T.ravel(), classifier_kPCA.predict(test_kPCA))

    print(f'error rate of kPCA on training data: {error_kPCA_train}')
    print(f'error rate of kPCA on testing data: {error_kPCA_test}')


Performing standard PCA...
error rate of PCA on training data: 0.0980392156862745
error rate of PCA on testing data: 0.23076923076923073


Performing Kernel PCA...
error rate of kPCA on training data: 0.38235294117647056
error rate of kPCA on testing data: 0.42307692307692313
